### Testing optimization of bcRead
To run this code, place the notebook in the autoPostProcessing folder
( you may need to pull most recent changes for the folder structures

In [5]:
from libs import bcRead
from libs.eqRead import eqRead

import rawpy
from rawpy import LibRawNonFatalError
import cv2
import numpy as np
#import piexif
import lensfunpy
import time
import timeit
from PIL import Image
import pandas as pd
from glob import glob

def openImageFile(imgPath, demosaic = rawpy.DemosaicAlgorithm.AHD):
    """ given an image path, attempts to return a numpy array image object """

    try:  # use rawpy to convert raw to openCV
        #https://letmaik.github.io/rawpy/api/rawpy.Params.html?highlight=demosaic%20algorithms
        with rawpy.imread(imgPath) as raw:
            bgr = raw.postprocess(chromatic_aberration=(1,1),
                                  demosaic_algorithm = demosaic) # a numpy RGB array
            im = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB) # the OpenCV image

    # if it is not a raw format, just try and open it.
    except LibRawNonFatalError:
        im = cv2.imread(imgPath)
    # convert cv2 format to rgb
    return im


#### Initiate the class, & pre-compile the regex

In [10]:
imgPath = './exampleImages/UCHT_test_set/UCHT_busy_noBlur_1BC.CR2'

prefix = "UCHT"
digits = 6
bcRead = bcRead.bcRead(prefix, digits)

#### Test preemptive grey scale conversion

In [51]:
# open the file
im = openImageFile(imgPath)

greyTimes = []
for i in range(0,20):
    startTime = time.time()
    grey = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    # resize the image for bc process
    # 1250, 1875
    # read the BC
    bc = bcRead.decodeBC(grey)
    bcs = len(bc)
    if bcs < 1:
        print('no bcFound')
    elapsedTime = time.time() - startTime
    greyTimes.append(elapsedTime)

greyScaleTimes = np.mean(greyTimes)
print(f'greyScale times averaged {greyScaleTimes}')

# open the file
im = openImageFile(imgPath)

times = []
for i in range(0,20):
    startTime = time.time()
    # resize the image for bc process
    # 1250, 1875
    # read the BC
    bc = bcRead.decodeBC(im)
    bcs = len(bc)
    if bcs < 1:
        print('no bcFound')
    #print(f'barcode(s) found: {bc}')
    elapsedTime = time.time() - startTime
    times.append(elapsedTime)

rgbScaleTimes = np.mean(times)
print(f'RGBscale times averaged {rgbScaleTimes}')

diff =  rgbScaleTimes - greyScaleTimes
print(f' converting to grey scale was: {diff} seconds faster')

greyScale times averaged 1.163443684577942
RGBscale times averaged 1.431900644302368
 converting to grey scale was: 0.2684569597244262 seconds faster


#### Converting to Greyscale is well worth the cost.

#### Test image size reduction

In [52]:
# open the file
im = openImageFile(imgPath)

unScaledTimes = []
for i in range(0,20):
    startTime = time.time()
    grey = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    # read the BC
    bc = bcRead.decodeBC(grey)
    bcs = len(bc)
    if bcs < 1:
        print('no bcFound')
    elapsedTime = time.time() - startTime
    unScaledTimes.append(elapsedTime)

print(grey.shape)
    
unScaledTime = np.mean(unScaledTimes)
print(f'unScaled times averaged {unScaledTime}')

# open the file
im = openImageFile(imgPath)

# .75 size of default large CR2 from UCHT equipment
# sizes ~ 60% seem to fail to find bc.

largeDim = 4347
smalldim = 2902

downScaledTimes = []
for i in range(0,20):
    startTime = time.time()
    grey = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    # resize the image for bc process
    h, w = grey.shape
    if w > h:
        w = largeDim
        h = smallDim   
    else:
        w = smallDim
        h = largeDim
    res = cv2.resize(grey,(w, h), interpolation = cv2.INTER_AREA)
    bc = bcRead.decodeBC(res)
    bcs = len(bc)
    if bcs < 1:
        print('no bcFound')
    elapsedTime = time.time() - startTime
    downScaledTimes.append(elapsedTime)

print(res.shape)
downScaledTime = np.mean(downScaledTimes)
print(f'Down Scaled times averaged {downScaledTime}')

diff = unScaledTime - downScaledTime
print(f' converting to grey scale was: {diff} seconds faster')

(3870, 5796)
unScaled times averaged 1.13950355052948
(2786, 4347)
Down Scaled times averaged 0.6824545860290527
 converting to grey scale was: 0.4570489645004272 seconds faster


#### down scaling and greyscaling appear to be worth the cost.

In [54]:
print(f'combined, greyscale and downscale costs {downScaledTime} seconds compared to {rgbScaleTimes}')
print(f'A {rgbScaleTimes - downScaledTime} difference')

combined, greyscale and downscale costs 0.6824545860290527 seconds compared to 1.431900644302368
A 0.7494460582733153 difference
